In [ ]:
# filepath: /Users/tomwattley/App/racing-api-project/racing-api-project/apps/trader/src/trader/market_trader.py
from dataclasses import dataclass
from datetime import datetime
import pandas as pd
import numpy as np
from api_helpers.clients import S3Client, BetFairClient
from api_helpers.clients.betfair_client import BetFairClient, BetFairOrder, OrderResult
from api_helpers.helpers.logging_config import I, W
from api_helpers.helpers.file_utils import S3FilePaths

import re


def print_dataframe_for_testing(df):
    print("pd.DataFrame({")

    for col in df.columns:
        value = df[col].iloc[0]
        if re.match(r"\d{4}-\d{2}-\d{2}", str(value)):
            str_test = (
                "[" + " ".join([f"pd.Timestamp('{x}')," for x in list(df[col])]) + "]"
            )
            print(f"'{col}':{str_test},")
        else:
            print(f"'{col}':{list(df[col])},")
    print("})")


class TestS3Client:
    def __init__(self):
        self.stored_data = None

    def store_data(self, data: pd.DataFrame, object_path: str):
        self.stored_data = {"object_path": object_path, "data": data}


class TestBetfairClient:
    def __init__(self):
        self.cash_out_market_ids = []
        self.placed_orders = []

    def cash_out_bets(self, market_ids: list[str]):
        self.cash_out_market_ids.append(list(market_ids))
        return self.cash_out_market_ids

    def place_order(self, betfair_order: BetFairOrder):
        self.placed_orders.append(betfair_order)
        return OrderResult(success=True, message="Test Bet Placed")


import pandas as pd


SELECTION_COLS = [
    "id",
    "timestamp",
    "race_id",
    "race_time",
    "race_date",
    "horse_id",
    "horse_name",
    "selection_type",
    "market_type",
    "market_id",
    "selection_id",
    "requested_odds",
    "valid",
    "invalidated_at",
    "invalidated_reason",
    "size_matched",
    "average_price_matched",
    "cashed_out",
    "fully_matched",
    "customer_strategy_ref",
    "processed_at",
]


class MaxStakeSizeExceededError(Exception):
    """Exception raised when the maximum stake size is exceeded."""

    def __init__(self, message="Maximum stake size exceeded."):
        self.message = message
        super().__init__(self.message)


@dataclass
class TradeRequest:
    valid_bets: bool
    info: str
    selections_data: pd.DataFrame | None = None
    orders: list[BetFairOrder] | None = None
    cash_out_market_ids: list[str] | None = None


class MarketTrader:
    def __init__(self, s3_client: S3Client, betfair_client: BetFairClient):
        self.s3_client = s3_client
        self.betfair_client = betfair_client
        self.paths = S3FilePaths()

    def trade_markets(
        self,
        stake_size: int,
        now_timestamp: pd.Timestamp,
        requests_data: pd.DataFrame,
    ) -> None:
        trades: TradeRequest = self._calculate_trade_positions(
            stake_size=stake_size,
            requests_data=requests_data,
            now_timestamp=now_timestamp,
        )

        if trades.cash_out_market_ids:
            self.betfair_client.cash_out_bets(trades.cash_out_market_ids)
            cash_out_data = (
                trades.selections_data[
                    trades.selections_data["market_id"].isin(trades.cash_out_market_ids)
                ]
                .assign(
                    cash_out_placed=True,
                    cashed_out_invalid=False,
                )
                .filter(
                    items=[
                        "market_id",
                        "selection_id",
                        "cash_out_placed",
                        "cashed_out_invalid",
                    ]
                )
            )
            trades.selections_data = (
                pd.merge(
                    trades.selections_data,
                    cash_out_data,
                    how="left",
                    on=["market_id", "selection_id"],
                )
                .assign(
                    cashed_out=lambda x: x["cash_out_placed"].fillna(x["cashed_out"]),
                    valid=lambda x: x["cashed_out_invalid"].fillna(x["valid"]),
                )
                .drop(columns=["cash_out_placed", "cashed_out_invalid"])
            )

        if trades.orders:
            for order in trades.orders:
                result: OrderResult = self.betfair_client.place_order(order)
                if result.success:
                    I(f"Order placed successfully: {order}")
                else:
                    W(f"Failed to place order: {order}, Error: {result.message}")

        if trades.selections_data is not None:
            self.s3_client.store_data(
                trades.selections_data[SELECTION_COLS],
                f"today/{now_timestamp.strftime('%Y_%m_%d')}/trader_data/selections.parquet",
            )

    def _calculate_trade_positions(
        self,
        stake_size: int,
        requests_data: pd.DataFrame,
        now_timestamp: pd.Timestamp,
    ) -> TradeRequest:
        upcoming_bets = self._check_bets_in_next_hour(requests_data)
        if upcoming_bets.empty:
            tr = TradeRequest(
                valid_bets=False,
                info="No bets in the next hour",
            )
            I(tr.info)
            return tr

        stake_exceeded = self._check_stake_size_exceeded(requests_data, stake_size)
        if not stake_exceeded.empty:
            raise MaxStakeSizeExceededError(
                f"Maximum stake size of {stake_size} exceeded for the following bets: {stake_exceeded}"
            )

        requests_data = (
            requests_data.pipe(self._mark_invalid_bets, now_timestamp)
            .pipe(self._mark_fully_matched_bets, stake_size, now_timestamp)
            .pipe(self._set_new_size_and_price, stake_size)
            .pipe(self._check_odds_available)
        )

        orders, cash_out_market_ids = self._create_bet_data(requests_data)
        selections_data = self._update_selections_data(requests_data)
        return TradeRequest(
            valid_bets=True,
            info="Bets requested!",
            selections_data=selections_data,
            orders=orders,
            cash_out_market_ids=cash_out_market_ids,
        )

    def _check_bets_in_next_hour(self, data: pd.DataFrame) -> pd.DataFrame:
        return data[(data["minutes_to_race"].between(0, 60))]

    def _check_stake_size_exceeded(
        self, data: pd.DataFrame, stake_size: int
    ) -> pd.DataFrame:
        data = data.assign(
            stake_exceeded=np.select(
                [
                    (data["selection_type"] == "BACK"),
                    (data["selection_type"] == "LAY"),
                ],
                [
                    (data["size_matched_betfair"] > stake_size),
                    (data["size_matched_betfair"] > stake_size * 1.5),
                ],
                default=False,
            )
        )

        return data[data["stake_exceeded"] == True]

    def _update_selections_data(self, data: pd.DataFrame) -> pd.DataFrame:
        data = data.assign(
            average_price_matched=data["average_price_matched_selections"]
            .fillna(data["average_price_matched_betfair"])
            .round(2),
            size_matched=data["size_matched_betfair"].round(2),
            customer_strategy_ref=data["customer_strategy_ref_selections"]
            .fillna(data["customer_strategy_ref_betfair"])
            .round(2),
        )
        return data.filter(items=SELECTION_COLS)

    def _mark_invalid_bets(
        self, data: pd.DataFrame, now_timestamp: pd.Timestamp
    ) -> pd.DataFrame:
        conditions = [
            (data["eight_to_seven_runners"] == True) & (data["market_type"] == "PLACE"),
            (data["short_price_removed_runners"] == True),
            (data["minutes_to_race"] < 1),
        ]

        return data.assign(
            valid=np.select(
                conditions,
                [False, False, False],
                default=data["valid"],
            ),
            invalidated_reason=np.select(
                conditions,
                ["Invalid 8 to 7 Place", "Invalid Short Price Removed", "Race Started"],
                default=data["invalidated_reason"],
            ),
            invalidated_at=np.select(
                conditions,
                [now_timestamp] * 3,
                default=data["invalidated_at"],
            ),
            processed_at=now_timestamp,
            cash_out=np.select(
                conditions,
                [True, True, False],
                default=False,
            ),
        )

    def _extract_invalidated_fully_matched_bets(
        self, data: pd.DataFrame, market_ids: list[str]
    ) -> pd.DataFrame:
        return data[(data["market_id"].isin(market_ids))]

    def _get_invalidated_fully_matched_bets_market_ids(
        self, data: pd.DataFrame
    ) -> list[str]:
        return (
            data[(data["valid"] == False) & (data["fully_matched"] == True)][
                "market_id"
            ]
            .unique()
            .tolist()
        )

    def _mark_fully_matched_bets(
        self, data: pd.DataFrame, stake_size: float, now_timestamp: pd.Timestamp
    ) -> pd.DataFrame:
        data = data.assign(
            staked_minus_target=np.select(
                [
                    (data["selection_type"] == "BACK"),
                    (data["selection_type"] == "LAY"),
                ],
                [
                    (stake_size - data["size_matched_betfair"]),
                    (
                        (stake_size * 1.5)
                        - (
                            data["size_matched_betfair"]
                            * (data["average_price_matched_betfair"] - 1)
                        )
                    ),
                ],
                default=0,
            )
        )
        data = data.assign(
            fully_matched=np.where(
                data["fully_matched"] == True,  # If already True, keep it True
                True,
                np.where(
                    data["staked_minus_target"] > 1, False, True
                ),  # Otherwise, calculate normally
            ),
            processed_at=now_timestamp,
        )

        return data

    def _set_new_size_and_price(
        self, data: pd.DataFrame, stake_size: float
    ) -> pd.DataFrame:
        conditions = [
            (data["selection_type"] == "BACK") & (data["size_matched_betfair"] > 0),
            (data["selection_type"] == "LAY") & (data["size_matched_betfair"] > 0),
            (data["selection_type"] == "BACK") & (data["size_matched_betfair"] == 0),
            (data["selection_type"] == "LAY") & (data["size_matched_betfair"] == 0),
        ]

        data = data.assign(
            remaining_size=np.select(
                conditions,
                [
                    stake_size - data["size_matched_betfair"],
                    (
                        (stake_size * 1.5)
                        - (data["average_price_matched_betfair"] - 1)
                        * data["size_matched_betfair"]
                    )
                    / (data["lay_price_1"] - 1),
                    stake_size,
                    (stake_size * 1.5) / (data["lay_price_1"] - 1),
                ],
            ),
            amended_average_price=np.select(
                conditions,
                [
                    (
                        (
                            (
                                data["average_price_matched_betfair"]
                                * data["size_matched_betfair"]
                            )
                            + (
                                data["back_price_1"]
                                * (stake_size - data["size_matched_betfair"])
                            )
                        )
                        / stake_size
                    ),
                    (
                        (stake_size * 1.5)
                        / (
                            (
                                (
                                    (stake_size * 1.5)
                                    - (data["average_price_matched_betfair"] - 1)
                                    * data["size_matched_betfair"]
                                )
                                / (data["lay_price_1"] - 1)
                            )
                            + data["size_matched_betfair"]
                        )
                        + 1
                    ).round(2),
                    data["back_price_1"],
                    data["lay_price_1"],
                ],
            ),
        )
        data = data.assign(
            remaining_size=data["remaining_size"].round(2),
            amended_average_price=data["amended_average_price"].round(2),
        )

        return data

    def _check_odds_available(self, data: pd.DataFrame) -> pd.DataFrame:
        return data.assign(
            available_odds=np.select(
                [
                    (data["selection_type"] == "BACK")
                    & (data["amended_average_price"] >= data["requested_odds"])
                    & (data["back_price_1_depth"] >= data["remaining_size"]),
                    (data["selection_type"] == "LAY")
                    & (data["amended_average_price"] <= data["requested_odds"])
                    & (data["lay_price_1_depth"] >= data["remaining_size"]),
                ],
                [True, True],
                default=False,
            )
        )

    def _create_bet_data(
        self, data: pd.DataFrame
    ) -> tuple[list[BetFairOrder], list[str]]:
        cash_out_market_ids = (
            data[data["cash_out"] == True]["market_id"].unique().tolist()
        )
        fully_matched_cash_out_market_ids = (
            data[(data["valid"] == False) & (data["fully_matched"] == True)][
                "market_id"
            ]
            .unique()
            .tolist()
        )

        bets = data[
            (data["valid"] == True)
            & (data["available_odds"] == True)
            & (data["cash_out"] == False)
            & (data["remaining_size"] > 1)
            & (data["fully_matched"] == False)
        ]

        orders = []

        for i in bets.itertuples():
            if i.selection_type == "BACK":
                order = BetFairOrder(
                    size=i.remaining_size,
                    price=i.back_price_1,
                    market_id=i.market_id,
                    selection_id=i.selection_id,
                    side=i.selection_type,
                    strategy="mvp",
                )
                orders.append(order)
            elif i.selection_type == "LAY":
                order = BetFairOrder(
                    size=i.remaining_size,
                    price=i.lay_price_1,
                    market_id=i.market_id,
                    selection_id=i.selection_id,
                    side=i.selection_type,
                    strategy="mvp",
                )
                orders.append(order)
            else:
                raise ValueError(f"Invalid selection type: {i.selection_type}")

        return orders, list(
            set(cash_out_market_ids + fully_matched_cash_out_market_ids)
        )


now_date_str = datetime.now().strftime("%Y-%m-%d")


def create_test_data(requests_overrides=None):
    requests_dict = {
        "id": ["1", "2", "3", "3"],
        "timestamp": [
            pd.Timestamp(f"{now_date_str} 12:00:00"),
            pd.Timestamp(f"{now_date_str} 17:00:00"),
            pd.Timestamp(f"{now_date_str} 18:00:00"),
            pd.Timestamp(f"{now_date_str} 18:00:00"),
        ],
        "race_id": [1, 2, 3, 3],
        "race_time": [
            pd.Timestamp(f"{now_date_str} 15:00:00"),
            pd.Timestamp(f"{now_date_str} 17:00:00"),
            pd.Timestamp(f"{now_date_str} 20:00:00"),
            pd.Timestamp(f"{now_date_str} 20:00:00"),
        ],
        "race_date": [
            pd.Timestamp(f"{now_date_str} 00:00:00"),
            pd.Timestamp(f"{now_date_str} 00:00:00"),
            pd.Timestamp(f"{now_date_str} 00:00:00"),
            pd.Timestamp(f"{now_date_str} 00:00:00"),
        ],
        "horse_id": [1, 2, 3, 3],
        "horse_name": ["Horse A", "Horse B", "Horse C", "Horse C"],
        "selection_type": ["BACK", "BACK", "BACK", "LAY"],
        "market_type": ["WIN", "WIN", "WIN", "WIN"],
        "market_id": ["1", "2", "3", "3"],
        "selection_id": [1, 2, 3, 3],
        "requested_odds": [3.0, 4.0, 7.0, 7.0],
        "valid": [True, True, False, False],
        "invalidated_at": [pd.NaT, pd.NaT, pd.NaT, pd.NaT],
        "invalidated_reason": ["", "", "", ""],
        "cashed_out": [False, False, True, True],
        "fully_matched": [False, False, False, False],
        "processed_at": [
            pd.Timestamp(f"{now_date_str} 12:00"),
            pd.Timestamp(f"{now_date_str} 17:00"),
            pd.Timestamp(f"{now_date_str} 18:00"),
            pd.Timestamp(f"{now_date_str} 18:00"),
        ],
        "minutes_to_race": [-10, 10, 20, 20],
        "back_price_1": [np.nan, 4.0, 7.0, 7.0],
        "back_price_1_depth": [np.nan, 100.0, 100.0, 100.0],
        "back_price_2": [np.nan, 4.8, 6.8, 6.8],
        "back_price_2_depth": [np.nan, 100.0, 100.0, 100.0],
        "lay_price_1": [np.nan, 5.2, 7.2, 7.2],
        "lay_price_1_depth": [np.nan, 100.0, 100.0, 100.0],
        "lay_price_2": [np.nan, 5.4, 7.4, 7.4],
        "lay_price_2_depth": [np.nan, 100.0, 100.0, 100.0],
        "eight_to_seven_runners": [False, False, False, False],
        "short_price_removed_runners": [False, False, False, False],
        "average_price_matched_selections": [np.nan, np.nan, np.nan, np.nan],
        "size_matched_selections": [0.0, 0.0, 0.0, 0.0],
        "customer_strategy_ref_selections": [
            "selection",
            "selection",
            "selection",
            "selection",
        ],
        "average_price_matched_betfair": [np.nan, np.nan, 7.0, 7.2],
        "size_matched_betfair": [0.0, 0.0, 10.0, 9.7],
        "customer_strategy_ref_betfair": [np.nan, np.nan, np.nan, np.nan],
    }

    return pd.DataFrame(
        {
            **requests_dict,
            **(requests_overrides or {}),
        }
    )

In [41]:
create_test_data()

,id,timestamp,race_id,race_time,race_date,horse_id,horse_name,selection_type,market_type,market_id,...,lay_price_2,lay_price_2_depth,eight_to_seven_runners,short_price_removed_runners,average_price_matched_selections,size_matched_selections,customer_strategy_ref_selections,average_price_matched_betfair,size_matched_betfair,customer_strategy_ref_betfair
0,1,2025-05-31 12:00:00,1,2025-05-31 15:00:00,2025-05-31,1,Horse A,BACK,WIN,1,...,NaN,NaN,False,False,NaN,0.0,selection,NaN,0.0,NaN
1,2,2025-05-31 17:00:00,2,2025-05-31 17:00:00,2025-05-31,2,Horse B,BACK,WIN,2,...,5.4,100.0,False,False,NaN,0.0,selection,NaN,0.0,NaN
2,3,2025-05-31 18:00:00,3,2025-05-31 20:00:00,2025-05-31,3,Horse C,BACK,WIN,3,...,7.4,100.0,False,False,NaN,0.0,selection,7.0,10.0,NaN
3,3,2025-05-31 18:00:00,3,2025-05-31 20:00:00,2025-05-31,3,Horse C,LAY,WIN,3,...,7.4,100.0,False,False,NaN,0.0,selection,7.2,9.7,NaN


In [36]:
now_timestamp = pd.Timestamp("2025-01-01 18:00:00")
stake_size = 10

s3_client = TestS3Client()
bf_client = TestBetfairClient()
trader = MarketTrader(
    s3_client=s3_client,
    betfair_client=bf_client,
)

trader.trade_markets(
    now_timestamp=now_timestamp,
    stake_size=stake_size,
    requests_data=create_test_data(),
)

2025-05-31T08:55:32Z | INFO - Order placed successfully: BetFairOrder(size=10.0, price=4.0, selection_id=2, market_id='2', side='BACK', strategy='mvp')


In [37]:
trader.s3_client.stored_data["object_path"]

'today/2025_01_01/trader_data/selections.parquet'

In [38]:
print_dataframe_for_testing(trader.s3_client.stored_data["data"])

pd.DataFrame({
'id':['1', '2', '3', '3'],
'timestamp':[pd.Timestamp('2025-05-31 12:00:00'), pd.Timestamp('2025-05-31 17:00:00'), pd.Timestamp('2025-05-31 18:00:00'), pd.Timestamp('2025-05-31 18:00:00'),],
'race_id':[1, 2, 3, 3],
'race_time':[pd.Timestamp('2025-05-31 15:00:00'), pd.Timestamp('2025-05-31 17:00:00'), pd.Timestamp('2025-05-31 20:00:00'), pd.Timestamp('2025-05-31 20:00:00'),],
'race_date':[pd.Timestamp('2025-05-31 00:00:00'), pd.Timestamp('2025-05-31 00:00:00'), pd.Timestamp('2025-05-31 00:00:00'), pd.Timestamp('2025-05-31 00:00:00'),],
'horse_id':[1, 2, 3, 3],
'horse_name':['Horse A', 'Horse B', 'Horse C', 'Horse C'],
'selection_type':['BACK', 'BACK', 'BACK', 'LAY'],
'market_type':['WIN', 'WIN', 'WIN', 'WIN'],
'market_id':['1', '2', '3', '3'],
'selection_id':[1, 2, 3, 3],
'requested_odds':[3.0, 4.0, 7.0, 7.0],
'valid':[False, True, True, True],
'invalidated_at':[pd.Timestamp('2025-01-01 18:00:00'), pd.Timestamp('NaT'), pd.Timestamp('NaT'), pd.Timestamp('NaT'),],
'invalidat

In [132]:
assert len(trader.s3_client.stored_data["data"]) == 3

In [133]:
trader.betfair_client.cash_out_market_ids

[['3', '2']]

In [108]:
trader.betfair_client.placed_orders

[]

- check size there 
- check odds avaiable 
- check handles partial bet updates 

In [48]:
# hack test
def run_test():
    """
    Test that the size available for betting is correctly calculated.

    """

    s3_client = TestS3Client()
    bf_client = TestBetfairClient()
    trader = MarketTrader(
        s3_client=s3_client,
        betfair_client=bf_client,
    )

    trader.trade_markets(
        now_timestamp=now_timestamp_fixture,
        stake_size=set_stake_size,
        requests_data=create_test_data(),
    )

    expected_selecions_data = pd.DataFrame(
        {
            "id": ["1", "2", "3"],
            "timestamp": [
                pd.Timestamp("2025-05-31 12:00:00"),
                pd.Timestamp("2025-05-31 17:00:00"),
                pd.Timestamp("2025-05-31 18:00:00"),
            ],
            "race_id": [1, 2, 3],
            "race_time": [
                pd.Timestamp("2025-05-31 15:00:00"),
                pd.Timestamp("2025-05-31 17:00:00"),
                pd.Timestamp("2025-05-31 20:00:00"),
            ],
            "race_date": [
                pd.Timestamp("2025-05-31 00:00:00"),
                pd.Timestamp("2025-05-31 00:00:00"),
                pd.Timestamp("2025-05-31 00:00:00"),
            ],
            "horse_id": [1, 2, 3],
            "horse_name": ["Horse A", "Horse B", "Horse C"],
            "selection_type": ["BACK", "LAY", "LAY"],
            "market_type": ["WIN", "WIN", "PLACE"],
            "market_id": ["1", "2", "3"],
            "selection_id": [1, 2, 3],
            "requested_odds": [3.0, 2.0, 3.0],
            "valid": [False, True, True],
            "invalidated_at": [
                pd.Timestamp("2025-01-01 18:00:00"),
                pd.Timestamp("NaT"),
                pd.Timestamp("NaT"),
            ],
            "invalidated_reason": ["Race Started", "", ""],
            "size_matched": [0.0, 5.0, 5.0],
            "average_price_matched": [np.nan, 2.0, 3.0],
            "cashed_out": [False, False, False],
            "fully_matched": [False, False, False],
            "customer_strategy_ref": ["selection", "selection", "selection"],
            "processed_at": [
                pd.Timestamp("2025-01-01 18:00:00"),
                pd.Timestamp("2025-01-01 18:00:00"),
                pd.Timestamp("2025-01-01 18:00:00"),
            ],
        }
    )
    expected_selecions_data["processed_at"] = expected_selecions_data[
        "processed_at"
    ].astype("datetime64[s]")

    # standard assertions
    assert len(trader.s3_client.stored_data["data"]) == 4
    assert (
        trader.s3_client.stored_data["object_path"]
        == "today/2025_01_01/trader_data/selections.parquet"
    )
    # standard assertions

    # pd.testing.assert_frame_equal(
    #     trader.s3_client.stored_data["data"],
    #     expected_selecions_data,
    # )
    # assert not trader.betfair_client.cash_out_market_ids
    print(trader.betfair_client.placed_orders)
    # assert not trader.betfair_client.placed_orders

    return (
        trader.s3_client.stored_data["data"],
        trader.s3_client.stored_data["object_path"],
        trader.betfair_client.cash_out_market_ids,
        trader.betfair_client.placed_orders,
    )


p = run_test()

2025-05-31T09:05:47Z | INFO - Order placed successfully: BetFairOrder(size=10.0, price=4.0, selection_id=2, market_id='2', side='BACK', strategy='mvp')


[BetFairOrder(size=10.0, price=4.0, selection_id=2, market_id='2', side='BACK', strategy='mvp')]


In [50]:
print_dataframe_for_testing(p[0])

pd.DataFrame({
'id':['1', '2', '3', '3'],
'timestamp':[pd.Timestamp('2025-05-31 12:00:00'), pd.Timestamp('2025-05-31 17:00:00'), pd.Timestamp('2025-05-31 18:00:00'), pd.Timestamp('2025-05-31 18:00:00'),],
'race_id':[1, 2, 3, 3],
'race_time':[pd.Timestamp('2025-05-31 15:00:00'), pd.Timestamp('2025-05-31 17:00:00'), pd.Timestamp('2025-05-31 20:00:00'), pd.Timestamp('2025-05-31 20:00:00'),],
'race_date':[pd.Timestamp('2025-05-31 00:00:00'), pd.Timestamp('2025-05-31 00:00:00'), pd.Timestamp('2025-05-31 00:00:00'), pd.Timestamp('2025-05-31 00:00:00'),],
'horse_id':[1, 2, 3, 3],
'horse_name':['Horse A', 'Horse B', 'Horse C', 'Horse C'],
'selection_type':['BACK', 'BACK', 'BACK', 'LAY'],
'market_type':['WIN', 'WIN', 'WIN', 'WIN'],
'market_id':['1', '2', '3', '3'],
'selection_id':[1, 2, 3, 3],
'requested_odds':[3.0, 4.0, 7.0, 7.0],
'valid':[False, True, True, True],
'invalidated_at':[pd.Timestamp('2025-01-01 18:00:00'), pd.Timestamp('NaT'), pd.Timestamp('NaT'), pd.Timestamp('NaT'),],
'invalidat

In [52]:
pd.DataFrame(
    {
        "id": ["1", "2", "3", "3"],
        "timestamp": [
            pd.Timestamp("2025-05-31 12:00:00"),
            pd.Timestamp("2025-05-31 17:00:00"),
            pd.Timestamp("2025-05-31 18:00:00"),
            pd.Timestamp("2025-05-31 18:00:00"),
        ],
        "race_id": [1, 2, 3, 3],
        "race_time": [
            pd.Timestamp("2025-05-31 15:00:00"),
            pd.Timestamp("2025-05-31 17:00:00"),
            pd.Timestamp("2025-05-31 20:00:00"),
            pd.Timestamp("2025-05-31 20:00:00"),
        ],
        "race_date": [
            pd.Timestamp("2025-05-31 00:00:00"),
            pd.Timestamp("2025-05-31 00:00:00"),
            pd.Timestamp("2025-05-31 00:00:00"),
            pd.Timestamp("2025-05-31 00:00:00"),
        ],
        "horse_id": [1, 2, 3, 3],
        "horse_name": ["Horse A", "Horse B", "Horse C", "Horse C"],
        "selection_type": ["BACK", "BACK", "BACK", "LAY"],
        "market_type": ["WIN", "WIN", "WIN", "WIN"],
        "market_id": ["1", "2", "3", "3"],
        "selection_id": [1, 2, 3, 3],
        "requested_odds": [3.0, 4.0, 7.0, 7.0],
        "valid": [False, True, True, True],
        "invalidated_at": [
            pd.Timestamp("2025-01-01 18:00:00"),
            pd.Timestamp("NaT"),
            pd.Timestamp("NaT"),
            pd.Timestamp("NaT"),
        ],
        "invalidated_reason": ["Race Started", "", "", ""],
        "size_matched": [0.0, 0.0, 10.0, 9.7],
        "average_price_matched": [np.nan, np.nan, 7.0, 7.2],
        "cashed_out": [False, False, True, True],
        "fully_matched": [False, False, True, True],
        "customer_strategy_ref": ["selection", "selection", "selection", "selection"],
        "processed_at": [
            pd.Timestamp("2025-01-01 18:00:00"),
            pd.Timestamp("2025-01-01 18:00:00"),
            pd.Timestamp("2025-01-01 18:00:00"),
            pd.Timestamp("2025-01-01 18:00:00"),
        ],
    }
)

,id,timestamp,race_id,race_time,race_date,horse_id,horse_name,selection_type,market_type,market_id,...,requested_odds,valid,invalidated_at,invalidated_reason,size_matched,average_price_matched,cashed_out,fully_matched,customer_strategy_ref,processed_at
0,1,2025-05-31 12:00:00,1,2025-05-31 15:00:00,2025-05-31,1,Horse A,BACK,WIN,1,...,3.0,False,2025-01-01 18:00:00,Race Started,0.0,NaN,False,False,selection,2025-01-01 18:00:00
1,2,2025-05-31 17:00:00,2,2025-05-31 17:00:00,2025-05-31,2,Horse B,BACK,WIN,2,...,4.0,True,NaT,,0.0,NaN,False,False,selection,2025-01-01 18:00:00
2,3,2025-05-31 18:00:00,3,2025-05-31 20:00:00,2025-05-31,3,Horse C,BACK,WIN,3,...,7.0,True,NaT,,10.0,7.0,True,True,selection,2025-01-01 18:00:00
3,3,2025-05-31 18:00:00,3,2025-05-31 20:00:00,2025-05-31,3,Horse C,LAY,WIN,3,...,7.0,True,NaT,,9.7,7.2,True,True,selection,2025-01-01 18:00:00


In [53]:
p[3]

[BetFairOrder(size=10.0, price=4.0, selection_id=2, market_id='2', side='BACK', strategy='mvp')]